In [1]:
import pandas as pd
import requests

# Load dataframes from TXT files
df_transcripts = pd.read_csv('output/grch37/compare_result_grch37.txt', sep='\t')
df_protein = pd.read_csv('protein_seq_crossmap_for_single_id.tsv', sep='\t')


/Users/lix2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/lix2/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/mw/_3xhfhq95wv51_k15b6h4cf4lj8gp1/T/ipykernel_77479/3099259.py:6: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_protein = pd.read_csv('protein_seq_crossmap_for_single_id.tsv', sep='\t')


In [2]:
# Function to call Ensembl VEP API to map NM IDs to ENST IDs
def get_enst_id(nm_id):
    server = "https://grch37.rest.ensembl.org"
    ext = f"/xrefs/symbol/homo_sapiens/{nm_id}?"
    headers = {"Content-Type": "application/json"}

    response = requests.get(server + ext, headers=headers)
    if not response.ok:
        return None

    data = response.json()
    for entry in data:
        if entry['id'].startswith('ENST'):
            return entry['id']
    return None

# Map NM IDs to ENST IDs via Ensembl API
def map_to_enst(x):
    if pd.isnull(x):
        return None
    x_str = str(x)
    if x_str.startswith("ENST"):
        return x_str
    return get_enst_id(x_str)

df_transcripts['MSK-Impact_enst_id'] = df_transcripts['MSK-Impact'].apply(map_to_enst)

In [3]:
def strip_version(enst_id):
    return enst_id.split('.')[0] if pd.notnull(enst_id) else None

protein_dict = {
    strip_version(row['grch37_enst_id']): {
        'sequence': row['protein_sequence'],
        'length': row['length']
    }
    for _, row in df_protein.iterrows()
}

In [4]:
def add_protein_info(df, col_name, new_col_prefix):
    sequences, lengths = [], []
    for enst in df[col_name]:
        if pd.notnull(enst) and str(enst).strip():
            enst_base = strip_version(enst)
            if enst_base in protein_dict:
                protein_info = protein_dict[enst_base]
                sequences.append(protein_info['sequence'])
                lengths.append(protein_info['length'])
            else:
                sequences.append(None)
                lengths.append(None)
        else:
            sequences.append(None)
            lengths.append(None)
    df[f'{new_col_prefix}_protein_sequence'] = sequences
    df[f'{new_col_prefix}_protein_length'] = lengths


# Add protein information for specified columns
for col in ['ensembl', 'oncokb', 'mskcc', 'MSK-Impact_enst_id']:
    add_protein_info(df_transcripts, col, col)

In [5]:
# Check for IDs without protein sequences and print them
for source in ['ensembl', 'oncokb', 'mskcc', 'MSK-Impact_enst_id']:
    missing_seq = df_transcripts[df_transcripts[f'{source}_protein_sequence'].isnull() & df_transcripts[source].notnull()]
    for _, row in missing_seq.iterrows():
        print(f"{source}: {row[source]}")

# Determine the longest protein and add new column
protein_length_cols = [
    'oncokb_protein_length',
    'mskcc_protein_length',
    'MSK-Impact_enst_id_protein_length'
]

def determine_longest_protein(row):
    lengths = [row[col] if pd.notnull(row[col]) else -1 for col in protein_length_cols]
    max_length = max(lengths)
    if lengths.count(max_length) == len(lengths):
        return "equal"
    sources = ['oncokb', 'mskcc', 'MSK-Impact']
    longest_sources = [sources[i] for i, length in enumerate(lengths) if length == max_length]
    if len(longest_sources) > 1:
        return ",".join(longest_sources)
    if max_length == -1:
        return None
    return longest_sources[0]

df_transcripts['longest_protein'] = df_transcripts.apply(determine_longest_protein, axis=1)

ensembl: ENST00000316450
ensembl: ENST00000344686
oncokb: ENST00000316450
oncokb: ENST00000344686
mskcc: ENST00000304494,ENST00000361570
mskcc: ENST00000607650
mskcc: ENST00000376926


In [6]:
def are_sequences_identical(row):
    seqs = [
        row.get('oncokb_protein_sequence'),
        row.get('mskcc_protein_sequence'),
        row.get('MSK-Impact_enst_id_protein_sequence')
    ]
    filtered = [s for s in seqs if s is not None]
    return all(s == filtered[0] for s in filtered)

df_transcripts['identical_protein_sequence'] = df_transcripts.apply(are_sequences_identical, axis=1)


In [7]:
!pip install rapidfuzz
from rapidfuzz.distance import Indel

In [8]:
def find_diff_ranges(base, other):
    if not base or not other:
        return None

    diffs = []
    for op, a0, a1, b0, b1 in Indel.opcodes(base, other):
        if op == 'equal':
            continue
        # give extra index for those first character is the same
        if a0 == 0 or a0 == 1:
            diffs.append(f"-{b1-1}-0")
        else:
            if a1 - 1 > a0:
                diffs.append(f"{a0}-{a1 - 1}")
            else:
                diffs.append(f"{a0}")

    return ",".join(diffs) if diffs else None

def compare_sequences(row, other_col):
    if row['identical_protein_sequence'] == True:
        return None
    base = row['oncokb_protein_sequence']
    other = row[other_col]
    return find_diff_ranges(base, other)

df_transcripts['oncokb_vs_mskcc'] = df_transcripts.apply(lambda row: compare_sequences(row, 'mskcc_protein_sequence'), axis=1)
df_transcripts['oncokb_vs_MSK-Impact'] = df_transcripts.apply(lambda row: compare_sequences(row, 'MSK-Impact_enst_id_protein_sequence'), axis=1)

In [9]:
df_transcripts.to_csv('output/grch37/TEMP_compare_result_grch37.txt', sep='\t', index=False)

In [10]:
# Print gene names where all lengths are equal but sequences differ
def sequences_are_equal(row):
    seqs = [
        row.get('ensembl_protein_sequence'),
        row.get('oncokb_protein_sequence'),
        row.get('mskcc_protein_sequence'),
        row.get('MSK-Impact_enst_id_protein_sequence')
    ]
    # Remove Nones and compare all non-null sequences to the first one
    filtered = [s for s in seqs if s is not None]
    return all(s == filtered[0] for s in filtered)

# Identify and print genes with length-equal but sequence-different
mismatch_rows = df_transcripts[df_transcripts['longest_protein'] == 'equal']
for _, row in mismatch_rows.iterrows():
    if not sequences_are_equal(row):
        print(f"Sequence mismatch despite equal length: {row['Hugo Symbol']}")


Sequence mismatch despite equal length: ABL1
Sequence mismatch despite equal length: BRCA1
Sequence mismatch despite equal length: CHEK2
Sequence mismatch despite equal length: CSF3R
Sequence mismatch despite equal length: CXCR4
Sequence mismatch despite equal length: CYLD
Sequence mismatch despite equal length: EIF4E
Sequence mismatch despite equal length: ERG
Sequence mismatch despite equal length: ETV1
Sequence mismatch despite equal length: FGFR2
Sequence mismatch despite equal length: FGFR3
Sequence mismatch despite equal length: FOXP1
Sequence mismatch despite equal length: FYN
Sequence mismatch despite equal length: GNAS
Sequence mismatch despite equal length: IKZF1
Sequence mismatch despite equal length: KRAS
Sequence mismatch despite equal length: MEN1
Sequence mismatch despite equal length: MET
Sequence mismatch despite equal length: MITF
Sequence mismatch despite equal length: MYD88
Sequence mismatch despite equal length: NF1
Sequence mismatch despite equal length: PAK1
Sequ